In [1]:
import hydra
import numpy as np
import pytorch_lightning as pl
import torch
from omegaconf import DictConfig, OmegaConf
from transformers import AutoTokenizer

from data import PunctuationDataModule, PunctuationInferenceDataset
import os
from models import PunctuationDomainModel

from nemo.utils.exp_manager import exp_manager
from time import time
from pytorch_lightning.callbacks import ModelCheckpoint

import atexit
from copy import deepcopy
import snoop
snoop.install()

from hydra.experimental import initialize, initialize_config_module, initialize_config_dir, compose
hydra.core.global_hydra.GlobalHydra.instance().clear()
initialize(config_path="project/experiment")
!pwd

cfg=compose(
    config_name="test_config.yaml", 
)
cfg.model.punct_label_ids=OmegaConf.create(sorted(cfg.model.punct_label_ids))
labels_to_ids = {_[1]:_[0] for _ in enumerate(cfg.model.punct_label_ids)}
ids_to_labels = {_[0]:_[1] for _ in enumerate(cfg.model.punct_label_ids)}

# model = PunctuationDomainModel.load_from_checkpoint(
#     checkpoint_path="/home/nxingyu2/project/Punctuation_with_Domain_discriminator/2021-02-06_17-15-23/checkpoints/Punctuation_with_Domain_discriminator-last.ckpt")

# trainer = pl.Trainer(**cfg.trainer)

/home/nxingyu2/project/experiment


In [11]:
queries = [
    'Hooooooo!',
    # 'what can i do for you today',
    # 'how are you',
    # 'good morning everyone how have your weekends been its a really great day'
]
cfg.model.punct_label_ids=OmegaConf.create(sorted(cfg.model.punct_label_ids))
labels_to_ids = {_[1]:_[0] for _ in enumerate(cfg.model.punct_label_ids)}
ds=PunctuationInferenceDataset(tokenizer=AutoTokenizer.from_pretrained('google/electra-small-discriminator'), queries=queries, max_seq_length=128, punct_label_ids=labels_to_ids, degree=0, attach_label_to_end=True)
ds[0]

{'input_ids': tensor([[ 101, 7570, 9541, 9541, 9541,  102,    0,    0,    0,    0,    0,    0,
             0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
             0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
             0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
             0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
             0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
             0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
             0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
             0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
             0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
             0,    0,    0,    0,    0,    0,    0,    0]]),
 'attention_mask': tensor([[ True,  True,  True,  True,  True,  True, False, False, False, False,
        

In [8]:

inference_results = model.add_punctuation(queries)
inference_results

[' we bought four shirts, one pen, and a mug from the nvidia gear store in santa clara                                                                                                            ',
 ' what can i do for you today?                                                                                                                        ',
 ' , how are you? ,                                                                                                                           ',
 ' good morning. everyone? how have your weekends been? its a really great day?                                                                                                                  ']